In [1]:
import os
import sys

#pyspark-shell
#os.system("pip install kafka-python")

os.environ["SPARK_HOME"]="/home/cloudera/Downloads/spark-2.4.4-bin-hadoop2.6"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
#os.environ["HADOOP_HOME"] ="/usr/lib/hadoop/bin"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
#os.environ["PYSPARK_PYTHON"] = "/usr/local/anaconda/bin/python" 
#os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/local/anaconda/bin/python"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

#org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.3 pyspark-shell'
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.4 pyspark-shell'


In [2]:
from pyspark.sql.types import TimestampType, StringType, StructType, StructField

# Path to our 20 JSON files
inputPath = "/home/cloudera/structured_streaming_data/"

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .appName("Spark Structured Streaming") \
  .getOrCreate()

In [4]:
# Explicitly set schema
schema = StructType([ StructField("id", StringType(), True),
                      StructField("first_name", StringType(), True),
                      StructField("last_name", StringType(), True),
                      StructField("email", StringType(), True),
                      StructField("gender", StringType(), True),
                      StructField("ip_address", StringType(), True)])

In [5]:
# Create DataFrame representing data in the JSON files
inputDF = (
  spark
    .read
    .schema(schema)
    .json(inputPath)
)

display(inputDF)

DataFrame[id: string, first_name: string, last_name: string, email: string, gender: string, ip_address: string]

In [6]:
# Remove empty rows
inputDF = inputDF.dropna()

display(inputDF)

DataFrame[id: string, first_name: string, last_name: string, email: string, gender: string, ip_address: string]

In [7]:
inputDF.collect()

[]

In [8]:
# Aggregate number of actions
actionsDF = (
  inputDF
    .groupBy(
       inputDF.gender
    )
    .count()
)


In [9]:
actionsDF.cache()

# Create temp table named 'iot_action_counts'
actionsDF.createOrReplaceTempView("students_data")

In [10]:
# Create streaming equivalent of `inputDF` using .readStream
streamingDF = (
  spark
    .readStream
    .schema(schema)
    .option("maxFilesPerTrigger", 1)
    .json(inputPath)
)

In [11]:
# Stream `streamingDF` while aggregating by action
streamingActionCountsDF = (
  streamingDF
    .groupBy(
      streamingDF.gender
    )
    .count()
)

In [12]:
# Is `streamingActionCountsDF` actually streaming?
streamingActionCountsDF.isStreaming

True

In [13]:
spark.conf.set("spark.sql.shuffle.partitions", "2")

# View stream in real-time
query = (
  streamingActionCountsDF
    .writeStream
    .format("memory")
    .queryName("counts")
    .outputMode("complete")
    .start()
)

In [ ]:
from IPython.display import display, clear_output
from time import sleep
while True:
    clear_output(wait=True)
    display(spark.sql('SELECT * FROM counts').show())
    sleep(1)

+------+-----+
|gender|count|
+------+-----+
|  null|    2|
|Female|   94|
|  Male|  104|
+------+-----+



None